In [34]:
import geopandas as gp
import pandas as pd
import xarray as xr
import rioxarray
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sb
import sys
from importlib import reload
sys.path.append('..')


In [18]:
HySpexPth = Path("/Volumes/FIREICE/fihyper/cwaigl/")
subpath = "03_products/cropped_masked_final/"
filepatt = "VNIR_SWIR_rad_geo_atm_bcor_crop.bsq"

In [3]:
fieldpath = Path("/Volumes/FIREICE/fiboreal/cwaigl/2021-HySpex_fuels/Field_Data") 
hex20 = "CW_hex20_BC_merged.gpkg"

In [6]:
hex20_df = gp.read_file(fieldpath / hex20)

In [9]:
hex20_df

,id,left,top,right,bottom,vegclass,flighline,fileprefix,layer,path,geometry
0,97562,437147.056349,7.182904e+06,437170.150360,7.182884e+06,Bare Ground,04,20200830-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((437268.300 7176543.741, 437274..."
1,8506,433509.749653,7.174824e+06,433532.843664,7.174804e+06,Closed Black/White Spruce Forest,01,20200830-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((433532.844 7174813.741, 433527..."
2,31053,434427.736581,7.175454e+06,434450.830592,7.175434e+06,Closed Tall Alder,01,20200830-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((434416.190 7175443.741, 434410..."
3,43399,434930.031316,7.175604e+06,434953.125326,7.175584e+06,Closed Spruce/Paper Birch/Aspen Forest,02,20200830-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((434935.805 7175603.741, 434930..."
4,129823,438446.094455,7.176694e+06,438469.188466,7.176674e+06,Black Spruce Woodland with Tussocks,05,20200830-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((438376.812 7176623.741, 438382..."
...,...,...,...,...,...,...,...,...,...,...,...
65,43399,434930.031316,7.175604e+06,434953.125326,7.175584e+06,Closed Spruce/Paper Birch/Aspen Forest,07,20210803-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((434935.805 7175603.741, 434930..."
66,123860,438203.607342,7.176674e+06,438226.701353,7.176654e+06,Open Black Spruce Forest,01,20210803-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((438244.022 7176653.741, 438238..."
67,109839,437632.030575,7.175924e+06,437655.124586,7.175904e+06,Wet Sedge Meadow,02,20210803-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((437447.278 7175813.741, 437441..."
68,57863,435518.928590,7.176004e+06,435542.022601,7.175984e+06,Closed Paper Birch Forest,06,20210803-BC,Dissolved,"MultiPolygon?crs=EPSG:32606&field=fid:long(0,0...","MULTIPOLYGON (((435553.570 7175993.741, 435559..."


In [13]:
hex20_df.groupby(by=['fileprefix', 'vegclass'])['vegclass'] .count()

fileprefix   vegclass                                
20200830-BC  Bare Ground                                 1
             Black Spruce Woodland with Tussocks         1
             Black Spruce/Tamarack Forest                1
             Closed Black Spruce Forest                  2
             Closed Black/White Spruce Forest            1
             Closed Paper Birch Forest                   3
             Closed Quaking Aspen Forest                 2
             Closed Quaking Aspen/White Spruce Forest    1
             Closed Spruce/Paper Birch Forest            2
             Closed Spruce/Paper Birch/Aspen Forest      1
             Closed Tall Alder                           2
             Closed Tall Birch/Willow Shrub              1
             Closed White Spruce Forest                  1
             Open Black Spruce Forest                    4
             Open Quaking Aspen/Spruce Forest            1
             Open Spruce/Paper Birch Forest              2
  

In [19]:
selected_band_idx = np.array([284,
 1,
 235,
 346,
 152,
 311,
 49,
 238,
 364,
 119,
 87,
 250,
 409,
 117,
 361,
 189,
 54,
 303,
 218,
 85,
 252,
 149,
 362,
 173,
 428,
 14,
 217,
 260,
 104,
 368,
 96,
 421,
 9,
 218,
 330,
 107,
 431,
 41,
 253,
 187,
 349,
 92,
 312,
 33,
 218,
 407,
 157,
 370,
 27,
 217])

In [22]:
def get_datafile(row, smooth=True):
    fpth = HySpexPth / f"{row.fileprefix.replace('-', '_')}" / subpath / f"{row.fileprefix}_{row.flightline}_{filepatt}"
    return fpth

def get_singlefilesampledf(labelrow, sampledf=None):
    return sampledf[(sampledf.fileprefix==labelrow.fileprefix) & (sampledf.flightline==labelrow.flightline)]

def get_dataset(fpth):
    testdata = xr.open_dataset(fpth, engine="rasterio")
    testdata.attrs['long_name'] = "spectral reflectance, %*100"
    testdata.attrs['name'] = "spectral reflectance, %*100"
    testdata['band_data'].attrs['long_name']  =  "spectral reflectance, %*100"
    return testdata

def get_spectra_fromsingle(samplerow, dataset='dummy'):
    pt = samplerow.geometry
    spectrum = dataset.band_data.sel(x=pt.x, y=pt.y, method='nearest').values[:-10]
    return spectrum

def get_spectra(labelrow, sampledf=None):
    results = []
    subsubsample = get_singlefilesampledf(labelrow, sampledf)
    fn = get_datafile(labelrow)
    print(f"working on {fn}")
    testdata = get_dataset(fn)
    print("Got dataset opened") 
    for idx, row in subsubsample.iterrows():
        print(f"   working on {idx}, {row.species} at {row.geometry}") 
        spectrum = get_spectra_fromsingle(row, dataset=testdata)
        dictitem = {
            'fid': idx,
            'species': row.species,
            'event': row.fileprefix,
            'flightline': row.flightline,
            'vegclass': row.vegclass,
            'context': row.context,
            'spectrum': spectrum,
            }
        results.append(dictitem)
    return results
    testdata.close()

In [14]:
vegpath = Path("/Volumes/FIREICE/fiboreal/cwaigl/2021-HySpex_fuels/GIS/veg_locations/") 
hex20_BC_2020 = "CW_20200830_hex_labeled.gpkg"

In [27]:
hex20_df = gp.read_file(vegpath / hex20_BC_2020)
hex20_df.rename(columns={'flighline': 'flightline'}, inplace=True)
hex20_df['fileprefix'] = '20200830-BC'
hex20_df

,id,left,top,right,bottom,vegclass,flightline,geometry,fileprefix
0,140522,438879.107157,7.175704e+06,438902.201168,7.175684e+06,Wetlands,06,"POLYGON ((438879.107 7175693.741, 438884.881 7...",20200830-BC
1,140524,438879.107157,7.175664e+06,438902.201168,7.175644e+06,Wetlands,06,"POLYGON ((438879.107 7175653.741, 438884.881 7...",20200830-BC
2,140098,438861.786649,7.175674e+06,438884.880660,7.175654e+06,Wetlands,06,"POLYGON ((438861.787 7175663.741, 438867.560 7...",20200830-BC
3,140523,438879.107157,7.175684e+06,438902.201168,7.175664e+06,Wetlands,06,"POLYGON ((438879.107 7175673.741, 438884.881 7...",20200830-BC
4,136258,438705.902076,7.175784e+06,438728.996087,7.175764e+06,Open Spruce/Paper Birch Forest,06,"POLYGON ((438705.902 7175773.741, 438711.676 7...",20200830-BC
...,...,...,...,...,...,...,...,...,...
130,139293,438827.145633,7.174734e+06,438850.239643,7.174714e+06,Water,06,"POLYGON ((438827.146 7174723.741, 438832.919 7...",20200830-BC
131,138421,438792.504617,7.175134e+06,438815.598627,7.175114e+06,Water,06,"POLYGON ((438792.505 7175123.741, 438798.278 7...",20200830-BC
132,139718,438844.466141,7.174744e+06,438867.560152,7.174724e+06,Water,06,"POLYGON ((438844.466 7174733.741, 438850.240 7...",20200830-BC
133,142210,438948.389189,7.176024e+06,438971.483200,7.176004e+06,Water,06,"POLYGON ((438948.389 7176013.741, 438954.163 7...",20200830-BC


In [28]:
hex20_df.flightline.unique()

array(['06', '05', '04', '03', '02', '01'], dtype=object)

In [68]:
labeldf = hex20_df[['fileprefix', 'flightline']].drop_duplicates()
samples = []
for _, labelrow in labeldf.iterrows():
    subsubsample = get_singlefilesampledf(labelrow, hex20_df)
    fn = get_datafile(labelrow)
    print(f"working on {fn}")

    for idx, row in subsubsample.iterrows():
        print(f"   working on {idx}, {row.vegclass} at {row.geometry}") 
        try:
            clipped = rioxarray.open_rasterio(fn, masked=True).rio.clip([row.geometry], from_disk=True)
        except ValueError:
            continue
        stacked = clipped.sel(band=(selected_band_idx[:-10] + 1)).stack(desired=['x', 'y'])
        dictelem = {
            'label': row.vegclass,
            'data_means': stacked.mean(axis=1).data,
            'data_stds': stacked.std(axis=1).data
        }
        samples.append(dictelem)

samples
                                     


working on /Volumes/FIREICE/fihyper/cwaigl/20200830_BC/03_products/cropped_masked_final/20200830-BC_06_VNIR_SWIR_rad_geo_atm_bcor_crop.bsq
   working on 0, Wetlands at POLYGON ((438879.10715690156 7175693.7406, 438884.88065959344 7175703.7406, 438896.42766497727 7175703.7406, 438902.20116766915 7175693.7406, 438896.42766497727 7175683.7406, 438884.88065959344 7175683.7406, 438879.10715690156 7175693.7406))
   working on 1, Wetlands at POLYGON ((438879.10715690156 7175653.7406, 438884.88065959344 7175663.7406, 438896.42766497727 7175663.7406, 438902.20116766915 7175653.7406, 438896.42766497727 7175643.7406, 438884.88065959344 7175643.7406, 438879.10715690156 7175653.7406))
   working on 2, Wetlands at POLYGON ((438861.7866488259 7175663.7406, 438867.5601515178 7175673.7406, 438879.1071569016 7175673.7406, 438884.8806595935 7175663.7406, 438879.1071569016 7175653.7406, 438867.5601515178 7175653.7406, 438861.7866488259 7175663.7406))
   working on 3, Wetlands at POLYGON ((438879.107156901

[{'label': 'Wetlands',
  'data_means': array([ 729.3908046 ,    6.65804598, 1488.15229885,  367.63505747,
         1031.63218391,  881.81321839,  193.96264368, 1425.96551724,
          377.13218391,  825.33333333,  247.81321839, 1066.18390805,
          495.73563218,  814.47988506,  294.29022989, 1515.3045977 ,
          206.01436782,  910.6637931 , 1369.59482759,  224.35344828,
          971.8908046 , 1048.91666667,  327.        , 1198.50574713,
          388.42528736,   40.6954023 , 1368.48850575,  695.74425287,
          694.09482759,  413.95689655,  484.09195402,  409.99137931,
           35.65229885, 1369.59482759,  609.46264368,  716.68678161,
          387.55172414,  169.25574713,  927.19827586, 1423.67528736]),
  'data_stds': array([ 87.01170569,  12.61320092, 227.56570939, 168.37440024,
         148.27290116, 114.11612934,  22.92218937, 214.82128088,
          97.35245127, 103.85416573,  22.33630486, 111.16889695,
          89.44926768, 100.92928643,  96.07728102, 241.30759519

In [70]:
pd.DataFrame.from_records(samples)

,label,data_means,data_stds
0,Wetlands,"[729.3908045977012, 6.658045977011494, 1488.15...","[87.0117056891705, 12.613200924472102, 227.565..."
1,Wetlands,"[1093.3304597701149, 9.494252873563218, 2164.7...","[108.01837057902338, 15.581726375969867, 133.3..."
2,Wetlands,"[940.3439306358382, 10.45664739884393, 1774.82...","[309.58422645931324, 16.190149873361754, 509.0..."
3,Wetlands,"[897.6408045977012, 9.85344827586207, 1779.066...","[88.3649385105502, 18.49258362233186, 196.4476..."
4,Open Spruce/Paper Birch Forest,"[1087.2260869565218, 15.443478260869565, 2335....","[186.3723588171403, 22.582868190904968, 288.76..."
...,...,...,...
127,Closed Black/White Spruce Forest,"[616.6521739130435, 11.710144927536232, 1600.0...","[198.16166918189617, 20.26155168763215, 420.50..."
128,Closed Black/White Spruce Forest,"[535.6224783861671, 2.276657060518732, 1352.06...","[166.71157118611964, 8.743452209974008, 359.24..."
129,Closed Tall Alder,"[806.1011560693642, 56.63583815028902, 1830.58...","[216.93653649331532, 41.4011154106442, 426.585..."
130,Closed Tall Alder,"[962.4092219020173, 70.98270893371757, 2116.81...","[269.19696939219443, 51.45205362331507, 535.13..."


In [77]:
columns_mean = [f"mean_{ii}" for ii in selected_band_idx[:-10] + 1] 
columns_std = [f"std_{ii}" for ii in selected_band_idx[:-10] + 1] 

In [82]:
X = pd.concat([pd.DataFrame(columns=columns_mean,
             data=[item['data_means'] for item in samples]), 
               pd.DataFrame(columns=columns_std,
             data=[item['data_stds'] for item in samples])], axis=1)

In [83]:
X

,mean_285,mean_2,mean_236,mean_347,mean_153,mean_312,mean_50,mean_239,mean_365,mean_120,...,std_97,std_422,std_10,std_219,std_331,std_108,std_432,std_42,std_254,std_188
0,729.390805,6.658046,1488.152299,367.635057,1031.632184,881.813218,193.962644,1425.965517,377.132184,825.333333,...,53.566187,85.753807,17.641621,213.570467,103.056236,88.819086,103.531689,21.472265,99.415437,226.033859
1,1093.330460,9.494253,2164.795977,258.135057,1358.675287,1318.945402,256.770115,2080.772989,557.505747,1045.359195,...,48.529004,102.963090,19.431725,111.962522,79.419517,86.769611,116.617711,20.457207,113.224961,100.702236
2,940.343931,10.456647,1774.823699,244.554913,1212.439306,1109.728324,217.072254,1713.265896,456.057803,978.263006,...,157.503907,218.485271,23.222991,471.666205,161.436334,229.959175,220.413038,55.909969,268.424853,456.255854
3,897.640805,9.853448,1779.066092,286.431034,1202.428161,1075.080460,213.882184,1705.330460,479.433908,928.114943,...,61.857955,94.775045,18.837549,178.536884,133.505840,103.770092,132.703219,30.635998,109.956190,170.489239
4,1087.226087,15.443478,2335.660870,223.197101,1696.150725,1294.014493,293.933333,2297.147826,429.472464,1538.739130,...,167.408657,124.314026,26.849773,252.358370,124.669177,296.855531,144.247815,60.165472,195.779765,242.310531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,616.652174,11.710145,1600.066667,160.095652,1488.359420,759.536232,236.466667,1512.933333,629.185507,1359.179710,...,311.678914,1371.446389,46.072197,376.010546,119.934790,548.952439,2221.003874,120.797915,175.059581,476.508980
128,535.622478,2.276657,1352.069164,197.538905,1168.302594,666.423631,174.132565,1278.319885,2580.072046,1039.827089,...,195.113700,4434.412329,26.036985,331.965645,114.799995,342.069636,4893.490965,68.839598,171.625452,369.592581
129,806.101156,56.635838,1830.586705,209.546243,1485.184971,978.919075,267.632948,1762.167630,466.184971,1344.054913,...,257.845888,800.305248,46.851463,393.148321,139.979372,436.365416,1359.200729,100.302487,199.602754,447.387692
130,962.409222,70.982709,2116.812680,246.634006,1616.204611,1159.622478,286.829971,2043.962536,449.386167,1470.746398,...,305.302329,157.573716,60.652563,493.657070,173.722245,524.996315,1114.923929,117.114149,269.149806,553.645893


In [84]:
y = pd.DataFrame(columns=['label'], data=[item['label'] for item in samples])
y

,label
0,Wetlands
1,Wetlands
2,Wetlands
3,Wetlands
4,Open Spruce/Paper Birch Forest
...,...
127,Closed Black/White Spruce Forest
128,Closed Black/White Spruce Forest
129,Closed Tall Alder
130,Closed Tall Alder


In [47]:
selected_band_idx + 1

array([285,   2, 236, 347, 153, 312,  50, 239, 365, 120,  88, 251, 410,
       118, 362, 190,  55, 304, 219,  86, 253, 150, 363, 174, 429,  15,
       218, 261, 105, 369,  97, 422,  10, 219, 331, 108, 432,  42, 254,
       188, 350,  93, 313,  34, 219, 408, 158, 371,  28, 218])

In [55]:
clipped.sel(band=(selected_band_idx[:-10] + 1)).mean(axis=2).mean(axis=1)

<xarray.DataArray (band: 40)>
array([1297.34054557,   12.30901894, 2343.11341326,  410.01651754,
       1461.37445927, 1520.38130222,  287.20172004, 2305.74801817,
        648.76315756, 1257.16016886,  278.04874206, 1699.66855997,
        923.00133847, 1249.98935092,  351.41429009, 2132.73090036,
        279.78757658, 1615.86781918, 2059.5885686 ,  249.47567136,
       1516.71615247, 1501.69290003,  464.72316078, 1751.39934831,
        677.77209995,   66.36924305, 2066.4484157 , 1012.43314718,
       1086.46239115,  770.17350744,  706.00839   ,  708.85556676,
         51.30644709, 2059.5885686 ,  908.92253866, 1133.65422778,
        676.25391839,  261.9207632 , 1429.461662  , 2021.65730848])
Coordinates:
    fwhm         (band) float64 6.52 3.8 6.52 6.52 3.8 ... 6.52 3.81 6.52 6.52
    wavelength   (band) float64 1.563e+03 413.5 ... 1.395e+03 1.036e+03
  * band         (band) int64 285 2 236 347 153 312 ... 331 108 432 42 254 188
    spatial_ref  int64 0

In [56]:
stacked = clipped.sel(band=(selected_band_idx[:-10] + 1)).stack(desired=['x', 'y'])

In [59]:
stacked.std(axis=1)

<xarray.DataArray (band: 40)>
array([348.62076157,  22.99526022, 581.42856718, 185.57809797,
       516.072795  , 405.48515295, 127.3238082 , 566.95897153,
       225.04923932, 438.55233549, 134.57746861, 394.58721914,
       265.17212104, 438.71351819, 118.55705632, 555.28105532,
       128.80771423, 423.10255055, 534.5480981 , 125.01078683,
       339.01014205, 530.629085  , 141.55878843, 478.66217489,
       201.55199575,  57.2886389 , 536.38246471, 227.72587883,
       387.88842364, 245.91277715, 256.81392504, 231.64317371,
        47.95688315, 534.5480981 , 202.05577748, 400.58238991,
       216.72485138, 117.54903961, 314.83413081, 528.95822488])
Coordinates:
    fwhm         (band) float64 6.52 3.8 6.52 6.52 3.8 ... 6.52 3.81 6.52 6.52
    wavelength   (band) float64 1.563e+03 413.5 ... 1.395e+03 1.036e+03
  * band         (band) int64 285 2 236 347 153 312 ... 331 108 432 42 254 188
    spatial_ref  int64 0